# KNN

In [1]:
from concrete import fhe
import numpy as np
from functools import reduce

from knn import knn

In [2]:
a = (np.random.rand(5) * 9).astype(int)
a

array([1, 0, 4, 5, 8])

In [3]:
knn(a)

array([1, 1, 0, 0, 0])

In [5]:
compiler = fhe.Compiler(knn, {"distances": "encrypted"})
circuit = compiler.compile([(a)])

In [7]:
circuit.encrypt_run_decrypt(a)

array([1, 1, 0, 0, 0], dtype=uint64)